In [ ]:
from brian2 import *
import matplotlib.pyplot as plt
import numpy as np

matplotlib.rcParams['axes.spines.top'] = False
matplotlib.rcParams['axes.spines.right'] = False
plt.rc('font', size=24)
plt.rc('axes', titlesize=24, labelsize=24)
plt.rc('xtick', labelsize=24)
plt.rc('ytick', labelsize=24)

In [ ]:
def run_all_currents(which):

    start_scope()
    # Default parameters
    tau_refr = (np.array([3,3,3,3])*ms)[which]
    v_stop = (np.array([30,30,30,30])*mV)[which]
    mem_cap = (np.array([200,200,100,100])*pfarad)[which]
    g_leak = (np.array([8,11,6,5])*nS)[which]
    e_rever = (np.array([-60,-70,-55,-57])*mV)[which]
    slope = (np.array([2.5,2.5,2.5,2.5])*mV)[which]
    v_thres = (np.array([-48,-44,-40,-40])*mV)[which]
    J_pot = (np.array([4,0,6,2.5])*nS)[which]
    J_spi = (np.array([85,150,25,20])*pA)[which]
    v_reset = (np.array([-42,-46,-57,-52])*mV)[which]
    tau_adapt = (np.array([200,200,50,100])*ms)[which]

    # x axis, constant currents
    currents_a = np.linspace(0,800,201)*pA
    currents_t = np.linspace(0,800,201)*pA
    currents_b = np.linspace(0,800,201)*pA
    currents_c = np.linspace(0,800,201)*pA
    currents = [currents_a,currents_t,currents_b,currents_c][which]

    # AdEx model
    adex = '''dv/dt = ( g_leak*(e_rever - v) + g_leak*slope*exp((v-v_thres)/slope) + curr_bg - curr_adapt)/mem_cap : volt (unless refractory)
                dcurr_adapt/dt = (-curr_adapt + J_pot*(v-e_rever))/tau_adapt : amp'''

    steps = len(currents)
    n_spikes = np.zeros(steps)
    first_isi = np.zeros(steps)
    last_isi = np.zeros(steps)
    ini_fr = np.zeros(steps)
    acco_ratio = np.zeros(steps)
    fr = np.zeros(steps)
    more_accurate_fr = np.zeros(steps)
    
    for i in range(steps):

        # Assign current value
        curr_bg = currents[i]
        # Single neuron
        neuron = NeuronGroup(1, model=adex, threshold='v > v_stop', reset='''v = v_reset
                                                                 curr_adapt = curr_adapt+J_spi''', refractory='tau_refr', method='euler', name='pop_a')
        neuron.curr_adapt = 0*amp
        neuron.v = e_rever
        smG = SpikeMonitor(neuron)
        rG = PopulationRateMonitor(neuron)
        v_mon = StateMonitor(neuron, 'v', record=True)
        adapt_mon = StateMonitor(neuron, 'curr_adapt', record=True)
        
        run_time = 500*ms
        run(run_time)
    
        n_spikes[i] = len(smG.t)
        # Spike-count firing rate
        fr[i] = n_spikes[i]*second/run_time
        
        if n_spikes[i]>=2:
            # ISI-based firing rate
            more_accurate_fr[i] = 1/mean(diff(smG.t/second))
            first_isi[i] = smG.t[1]-smG.t[0]
            # Initial firing rate
            ini_fr[i] = 1/(first_isi[i])
            
    return fr, more_accurate_fr, currents, ini_fr


# Same as above, but for just one current
def run_one_current(which, which_current):

    start_scope()
    tau_refr = (np.array([3,3,3,3])*ms)[which]
    v_stop = (np.array([-30,-30,-30,-30])*mV)[which]
    mem_cap = (np.array([200,200,100,100])*pfarad)[which]
    g_leak = (np.array([8,11,6,5])*nS)[which]
    e_rever = (np.array([-60,-70,-55,-57])*mV)[which]
    slope = (np.array([2.5,2.5,2.5,2.5])*mV)[which]
    v_thres = (np.array([-48,-44,-40,-40])*mV)[which]
    J_pot = (np.array([4,0,6,2.5])*nS)[which]
    J_spi = (np.array([85,150,25,20])*pA)[which]
    v_reset = (np.array([-42,-46,-57,-52])*mV)[which]
    tau_adapt = (np.array([200,200,50,100])*ms)[which]

    curr_bg = which_current*pA
    
    adex = '''dv/dt = ( g_leak*(e_rever - v) + g_leak*slope*exp((v-v_thres)/slope) + curr_bg - curr_adapt)/mem_cap : volt (unless refractory)
              dcurr_adapt/dt = (-curr_adapt + J_pot*(v-e_rever))/tau_adapt : amp'''
    
    neuron = NeuronGroup(1, model=adex, threshold='v > v_stop', reset='''v = v_reset
                                                             curr_adapt = curr_adapt+J_spi''', refractory='tau_refr', method='euler', name='neuron')
    neuron.curr_adapt = 0*amp
    neuron.v = e_rever
    smG = SpikeMonitor(neuron)
    rG = PopulationRateMonitor(neuron)
    v_mon = StateMonitor(neuron, 'v', record=True)
    adapt_mon = StateMonitor(neuron, 'curr_adapt', record=True)

    run_time = 500*ms
    run(run_time)
        
    potentials = v_mon.v[0]
    adaptations = adapt_mon.curr_adapt[0]/pA
    times = v_mon.t
    spike_indeces = (smG.t*10/ms).astype(int)
    potentials[spike_indeces] = 10*mV
       
    return potentials, adaptations, times, smG.t

In [ ]:
fr_a, ma_fr_a, currents_a, ini_fr_a = run_all_currents(0)
fr_t, ma_fr_t, currents_t, ini_fr_t = run_all_currents(1)
fr_b, ma_fr_b, currents_b, ini_fr_b = run_all_currents(2)
fr_c, ma_fr_c, currents_c, ini_fr_c = run_all_currents(3)

In [ ]:
# Possible transient firing
trans_t = np.where((fr_t<4) & (fr_t>0))
trans_a = np.where((fr_a<7) & (fr_a>0))
trans_b = np.where((fr_b<7) & (fr_b>0))
trans_c = np.where((fr_c<7) & (fr_c>0))
# Peristent firing or zero
per_t = np.where((fr_t>=4) | (fr_t==0))
per_a = np.where((fr_a>=7) | (fr_a==0))
per_b = np.where((fr_b>=7) | (fr_b==0))
per_c = np.where((fr_c>=4) | (fr_c==0))
# Use spike-count rate when there is suspicion of transient firing (set ISI-rate to nan)
ma_fr_t[trans_t]= NaN
ma_fr_a[trans_a]= NaN
ma_fr_b[trans_b]= NaN
ma_fr_c[trans_c]= NaN
# Use ISI-rate when there is no suspicion of transient firing (set spike-count rate to nan)
fr_t[per_t]=NaN
fr_a[per_a]=NaN
fr_b[per_b]=NaN
fr_c[per_c]=NaN
# For particularly 
#insuf_data_t=np.where((fr_t<4) & (fr_t>0))
#ma_fr_t[insuf_data_t]= fr_t[insuf_data_t]
currents_a=currents_a*10**12
currents_t=currents_t*10**12
currents_b=currents_b*10**12
currents_c=currents_c*10**12

In [ ]:
# Run samples for the inset plots
potentials_a, adaptations_a, times_a, aa = run_one_current(0,220)
potentials_t, adaptations_t, times_t, bb = run_one_current(1,550)
potentials_b, adaptations_b, times_b, cc = run_one_current(2,300)
potentials_c, adaptations_c, times_c, dd = run_one_current(3,200)

In [ ]:
np.savetxt('samples/single_neuron_traces/fr_a.csv', fr_a, delimiter=',')
np.savetxt('samples/single_neuron_traces/fr_t.csv', fr_t, delimiter=',')
np.savetxt('samples/single_neuron_traces/fr_b.csv', fr_b, delimiter=',')
np.savetxt('samples/single_neuron_traces/fr_c.csv', fr_c, delimiter=',')
np.savetxt('samples/single_neuron_traces/ma_fr_a.csv', ma_fr_a, delimiter=',')
np.savetxt('samples/single_neuron_traces/ma_fr_t.csv', ma_fr_t, delimiter=',')
np.savetxt('samples/single_neuron_traces/ma_fr_b.csv', ma_fr_b, delimiter=',')
np.savetxt('samples/single_neuron_traces/ma_fr_c.csv', ma_fr_c, delimiter=',')
np.savetxt('samples/single_neuron_traces/currents_a.csv', currents_a, delimiter=',')
np.savetxt('samples/single_neuron_traces/currents_t.csv', currents_t, delimiter=',')
np.savetxt('samples/single_neuron_traces/currents_b.csv', currents_b, delimiter=',')
np.savetxt('samples/single_neuron_traces/currents_c.csv', currents_c, delimiter=',')

np.savetxt('samples/single_neuron_traces/potentials_a.csv', potentials_a, delimiter=',')
np.savetxt('samples/single_neuron_traces/potentials_t.csv', potentials_t, delimiter=',')
np.savetxt('samples/single_neuron_traces/potentials_b.csv', potentials_b, delimiter=',')
np.savetxt('samples/single_neuron_traces/potentials_c.csv', potentials_c, delimiter=',')
np.savetxt('samples/single_neuron_traces/adaptations_a.csv', adaptations_a, delimiter=',')
np.savetxt('samples/single_neuron_traces/adaptations_t.csv', adaptations_t, delimiter=',')
np.savetxt('samples/single_neuron_traces/adaptations_b.csv', adaptations_b, delimiter=',')
np.savetxt('samples/single_neuron_traces/adaptations_c.csv', adaptations_c, delimiter=',')
np.savetxt('samples/single_neuron_traces/times_a.csv', times_a, delimiter=',')
np.savetxt('samples/single_neuron_traces/times_t.csv', times_t, delimiter=',')
np.savetxt('samples/single_neuron_traces/times_b.csv', times_b, delimiter=',')
np.savetxt('samples/single_neuron_traces/times_c.csv', times_c, delimiter=',')

In [ ]:
curr_a_linaro = np.array([50,75,100,125,150,175,200,225,250]) # from Linaro 2022
a_linaro_fr = np.array([0,4.4,8.72,10.53,12.44,14.3,16.64,20.4,25.36]) # from Linaro 2022
curr_t_linaro = np.array([100,150,200,250,300,350,400,450,500,550,600,650,700,750,800]) # from Linaro 2022
t_linaro_fr = np.array([0,0,0,3.8,5.8,8.4,9.6,10.8,12,13,14.6,16.2,18.4,21.2,24.8]) # from Linaro 2022
curr_b_nikolaus = np.array([0,40,80,120,160,200,240,280,320,360,400,440,480,520,560,600,640,680,720,760,800]) # from Fidzinski 2015
b_nikolaus_fr = np.array([0,0,0,0,25,40,48,55,62,69,75,88,91,88,100,100,113,115,113,125,125]) # from Fidzinski 2015

fr_a = np.genfromtxt('samples/single_neuron_traces/fr_a.csv', delimiter=',')
fr_t = np.genfromtxt('samples/single_neuron_traces/fr_t.csv', delimiter=',')
fr_b = np.genfromtxt('samples/single_neuron_traces/fr_b.csv', delimiter=',')
fr_c = np.genfromtxt('samples/single_neuron_traces/fr_c.csv', delimiter=',')
ma_fr_a = np.genfromtxt('samples/single_neuron_traces/ma_fr_a.csv', delimiter=',')
ma_fr_t = np.genfromtxt('samples/single_neuron_traces/ma_fr_t.csv', delimiter=',')
ma_fr_b = np.genfromtxt('samples/single_neuron_traces/ma_fr_b.csv', delimiter=',')
ma_fr_c = np.genfromtxt('samples/single_neuron_traces/ma_fr_c.csv', delimiter=',')
currents_a = np.genfromtxt('samples/single_neuron_traces/currents_a.csv', delimiter=',')
currents_t = np.genfromtxt('samples/single_neuron_traces/currents_t.csv', delimiter=',')
currents_b = np.genfromtxt('samples/single_neuron_traces/currents_b.csv', delimiter=',')
currents_c = np.genfromtxt('samples/single_neuron_traces/currents_c.csv', delimiter=',')

potentials_a = 1000*np.genfromtxt('samples/single_neuron_traces/potentials_a.csv', delimiter=',')
potentials_t = 1000*np.genfromtxt('samples/single_neuron_traces/potentials_t.csv', delimiter=',')
potentials_b = 1000*np.genfromtxt('samples/single_neuron_traces/potentials_b.csv', delimiter=',')
potentials_c = 1000*np.genfromtxt('samples/single_neuron_traces/potentials_c.csv', delimiter=',')
adaptations_a = np.genfromtxt('samples/single_neuron_traces/adaptations_a.csv', delimiter=',')
adaptations_t = np.genfromtxt('samples/single_neuron_traces/adaptations_t.csv', delimiter=',')
adaptations_b = np.genfromtxt('samples/single_neuron_traces/adaptations_b.csv', delimiter=',')
adaptations_c = np.genfromtxt('samples/single_neuron_traces/adaptations_c.csv', delimiter=',')
times_a = np.genfromtxt('samples/single_neuron_traces/times_a.csv', delimiter=',')
times_t = np.genfromtxt('samples/single_neuron_traces/times_t.csv', delimiter=',')
times_b = np.genfromtxt('samples/single_neuron_traces/times_b.csv', delimiter=',')
times_c = np.genfromtxt('samples/single_neuron_traces/times_c.csv', delimiter=',')

In [ ]:
connections = np.arange(1,26,1)
timeline = np.arange(2,2.5,0.0001)
timelinelong = np.arange(3,13,0.0001)

offset = 10

fig = plt.figure(figsize=(18,15))
ax11 = fig.add_subplot(2,2,1)
ax11.plot(currents_a,ma_fr_a,color='goldenrod',linewidth=2)
ax11.plot(currents_a,fr_a/2,linestyle='--',color='goldenrod',linewidth=2)
ax11.plot(curr_a_linaro,a_linaro_fr,'o',color='k',markersize=10)
ax11.axvline(x=220, ymin=(ma_fr_a[55]-offset)/120, ymax=(ma_fr_a[55]+offset)/120, color='gray',linewidth=2)
ax11.set_xlim([0, 600])
ax11.set_ylim([0,120])
ax11.set_xticks([100,300,500])
ax11.set_yticks([50,100])
ax11.set_xlabel('Injected current [pA]')
ax11.set_ylabel('Firing rate [spikes/s]')
ax11.set_title('Athorny (A)')

ax12 = ax11.inset_axes([0.15, 0.65, 0.4, 0.3])
ax120 = ax12.twinx()
ax12.set_xlim([0, 0.5])
ax12.set_ylim([-120,0])
ax120.set_ylim([0,750])
ax12.set_yticks([-50,0])
ax120.set_yticks([100,300])
ax12.plot(times_a,potentials_a,color='black')
ax120.plot(times_a,adaptations_a,color='red')
ax12.set_xlabel('Time [s]',fontsize=15)
ax12.set_ylabel('Potential [mV]', color='k',fontsize=15)
ax120.set_ylabel('Adaptation [pA]', color='r',fontsize=15)
ax120.spines['right'].set_visible(True)
ax120.spines['right'].set_color('red')
ax120.tick_params(axis='y', colors='red')
ax12.tick_params(axis='both', which='major', labelsize=15)
ax120.tick_params(axis='both', which='major', labelsize=15)

ax21 = fig.add_subplot(2,2,2)
ax21.plot(currents_a,ma_fr_t,color='magenta',linewidth=2)
ax21.plot(currents_a,fr_t/2,linestyle='--',color='magenta',linewidth=2)
ax21.plot(curr_t_linaro,t_linaro_fr,'o',color='black',markersize=10)
ax21.axvline(x=550, ymin=(ma_fr_t[138]-offset)/120, ymax=(ma_fr_t[138]+offset)/120, color='gray',linewidth=2)
ax21.set_xlim([0, 600])
ax21.set_ylim([0,120])
ax21.set_xticks([100,300,500])
ax21.set_yticks([50,100])
ax21.set_xlabel('Injected current [pA]')
ax21.set_ylabel('Firing rate [spikes/s]')
ax21.set_title('Thorny (T)')

ax22 = ax21.inset_axes([0.15, 0.65, 0.4, 0.3])
ax220 = ax22.twinx()
ax22.set_xlim([0, 0.5])
ax22.set_ylim([-120,0])
ax220.set_ylim([0,750])
ax22.set_yticks([-50,0])
ax220.set_yticks([100,300])
ax22.plot(times_t,potentials_t,color='black')
ax220.plot(times_t,adaptations_t,color='red')
ax22.set_xlabel('Time [s]',fontsize=15)
ax22.set_ylabel('Potential [mV]', color='k',fontsize=15)
ax220.set_ylabel('Adaptation [pA]', color='r',fontsize=15)
ax220.spines['right'].set_visible(True)
ax220.spines['right'].set_color('red')
ax220.tick_params(axis='y', colors='red')
ax22.tick_params(axis='both', which='major', labelsize=15)
ax220.tick_params(axis='both', which='major', labelsize=15)

ax31=fig.add_subplot(2,2,3)
ax31.plot(currents_b,ma_fr_b,'blue',linewidth=2)
ax31.plot(currents_b,fr_b/2,'b--',linewidth=2)
ax31.plot(curr_b_nikolaus,b_nikolaus_fr,'o',color='black',markersize=10)
ax31.axvline(x=300, ymin=(ma_fr_b[75]-offset)/120, ymax=(ma_fr_b[75]+offset)/120, color='gray',linewidth=2)
ax31.set_xlim([0, 600])
ax31.set_ylim([0,120])
ax31.set_xticks([100,300,500])
ax31.set_yticks([50,100])
ax31.set_xlabel('Injected current [pA]')
ax31.set_ylabel('Firing rate [spikes/s]')
ax31.set_title('PV-Basket (B)')

ax32 = ax31.inset_axes([0.15, 0.65, 0.4, 0.3])
ax320 = ax32.twinx()
ax32.set_xlim([0, 0.5])
ax32.set_ylim([-120,0])
ax320.set_ylim([0,300])
ax32.set_yticks([-50,0])
ax320.set_yticks([50,100])
ax32.plot(times_b,potentials_b,color='black')
ax320.plot(times_b,adaptations_b,color='red')
ax32.set_xlabel('Time [s]',fontsize=15)
ax32.set_ylabel('Potential [mV]', color='k',fontsize=15)
ax320.set_ylabel('Adaptation [pA]', color='r',fontsize=15)
ax320.spines['right'].set_visible(True)
ax320.spines['right'].set_color('red')
ax320.tick_params(axis='y', colors='red')
ax32.tick_params(axis='both', which='major', labelsize=15)
ax320.tick_params(axis='both', which='major', labelsize=15)

ax41 = fig.add_subplot(2,2,4)
ax41.plot(currents_c,ma_fr_c,'green',linewidth=2)
ax41.plot(currents_c,fr_c/2,'g--',linewidth=2)
ax41.axvline(x=200, ymin=(ma_fr_c[50]-offset)/120, ymax=(ma_fr_c[50]+offset)/120, color='gray',linewidth=2)
ax41.set_xlim([0, 600])
ax41.set_ylim([0,120])
ax41.set_xticks([100,300,500])
ax41.set_yticks([50,100])
ax41.set_xlabel('Injected current [pA]')
ax41.set_ylabel('Firing rate [spikes/s]')
ax41.set_title('Anti-SPW (C)')

ax42 = ax41.inset_axes([0.15, 0.65, 0.4, 0.3])
ax420 = ax42.twinx()
ax42.set_xlim([0, 0.5])
ax42.set_ylim([-120,0])
ax420.set_ylim([0,250])
ax42.set_yticks([-50,0])
ax420.set_yticks([50,100])
ax42.plot(times_c,potentials_c,color='black')
ax420.plot(times_c,adaptations_c,color='red')
ax42.set_xlabel('Time [s]',fontsize=15)
ax42.set_ylabel('Potential [mV]', color='k',fontsize=15)
ax420.set_ylabel('Adaptation [pA]', color='r',fontsize=15)
ax420.spines['right'].set_visible(True)
ax420.spines['right'].set_color('red')
ax420.tick_params(axis='y', colors='red')
ax42.tick_params(axis='both', which='major', labelsize=15)
ax420.tick_params(axis='both', which='major', labelsize=15)

plt.tight_layout()
plt.savefig('Fig4-figure supplement 1.jpg',dpi=300)
plt.show()